In [1]:
import geopandas as gpd
import cobra.postgres.interface as pgi
import cobra.utils.filehandling as fh
import cobra.utils.jobhandling as jh
import geopandas as gpd
import cobra.utils.geodatahandling as gdh

In [2]:
fileman = fh.Filemanager()
jobman = jh.Jobmanager()

In [ ]:
#jobman.delete_jobs()

In [ ]:
#fileman.download_plain_file('https://download.geofabrik.de/europe/norway-latest.osm.pbf','Norway')

In [ ]:
! ls /download/Norway

In [ ]:
datasets = fileman.get_data(datatype='OSM PBF')
datasets.head()

In [ ]:
norway = datasets.iloc[0]

In [ ]:
norway

In [ ]:
jobman.create_import_job_from_dataset(norway, 'norwaypower', style='/styles/power.style')

In [ ]:
! ls /styles

In [ ]:
jobman.get_jobs(df=True)

In [ ]:
#sql = 'SELECT osm.way AS geom, osm.* FROM man2.planet_osm_line osm WHERE power IS NOT NULL'
sql = "SELECT osm.way AS geom, osm.* FROM man2.planet_osm_line osm WHERE power IS NOT NULL AND operator like 'Lede'"
pg_interface = pgi.PgInterface()
conn = pg_interface.get_connection()
powerlines = gpd.GeoDataFrame.from_postgis(sql, conn)

In [ ]:
powerlines.head()

In [ ]:
powerlines.plot(figsize=(16, 16))

In [9]:
sql = "SELECT osm.way AS geom, osm.* FROM man2.planet_osm_line osm WHERE power = 'line'"
jobman.create_new_pg_to_x(sql=sql, format='GPKG', filename='norway_powerline.gpkg')

In [10]:
jobman.get_jobs(df=True)

,UUID,Name,Job Type,Date created,Date started,Date finished,Priority,Status
0,c0f11530-3600-419a-94e3-4318ff041bfd,Export PG,pg2x,"Thu, 20 Jan 2022 20:38:12 GMT",None,None,42,Failed - no details
1,7bfe3a33-c173-4adb-99ae-5d2600ff6f10,Export PG,pg2x,"Thu, 20 Jan 2022 22:41:16 GMT",None,"Thu, 20 Jan 2022 22:41:22 GMT",42,Finished
2,0ecbc056-c33b-4203-9a46-1fee0e28305f,Export PG,pg2x,"Thu, 20 Jan 2022 22:44:22 GMT",None,"Thu, 20 Jan 2022 22:44:23 GMT",42,Finished


In [ ]:
jobman.delete_jobs()

In [11]:
! ls /export -l

total 6208
-rw-r--r-- 1 root root  344064 Jan 20 22:41 lede3.gpkg
-rw-r--r-- 1 root root 6012928 Jan 20 22:44 norway_powerline.gpkg
